results 4.3.2

In [1]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import datetime as dt
from random import seed
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K
tf.compat.v1.experimental.output_all_intermediates(True)

In [2]:
tf.random.set_seed(36)

## Dataset

In [3]:
BASE_DIR_PATH = '/Users/neilb/Documents/dsci_thesis/Datasets'
DATASET_FILE = os.path.join(BASE_DIR_PATH, 'compiled_data_2016_2017.csv')

In [4]:
df = pd.read_csv(DATASET_FILE)
df

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x
0,0,0,0,1,2,0,0,12.18,21.03,21.033407,14.842428,803.88,630.9,0.026165,0.023526,2016-01-01 00:00:00,0.0,14420
1,1,1,0,1,1,1,0,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 01:00:00,3600.0,14420
2,2,2,1,1,1,0,1,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 02:00:00,7200.0,14420
3,3,3,0,0,0,1,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 03:00:00,10800.0,14420
4,4,4,1,1,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 04:00:00,14400.0,14420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,16059,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 19:00:00,63140400.0,14420
17516,17516,16424,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 20:00:00,63144000.0,14420
17517,17517,16789,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 21:00:00,63147600.0,14420
17518,17518,17154,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 22:00:00,63151200.0,14420


In [5]:
df = df[['x', 't', 'Discharge_Sto_Nino', 'Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']]
df.head()

,x,t,Discharge_Sto_Nino,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Velocity_Sto_Nino
0,14420,0.0,21.033407,0,1,2,0,0,12.18,0.026165
1,14420,3600.0,21.280072,0,1,1,1,0,12.19,0.026450
2,14420,7200.0,21.280072,1,1,1,0,1,12.19,0.026450
3,14420,10800.0,21.529056,0,0,0,1,0,12.20,0.026738
4,14420,14400.0,21.529056,1,1,1,0,0,12.20,0.026738


In [6]:
n = len(df)

In [7]:
df['friction_coeff'] = [0.033 for i in range(n)]
df['slope'] = [1/1500 for i in range(n)]
df.head()

C:\Users\neilb\AppData\Local\Temp\ipykernel_12824\1076625494.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['friction_coeff'] = [0.033 for i in range(n)]
C:\Users\neilb\AppData\Local\Temp\ipykernel_12824\1076625494.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['slope'] = [1/1500 for i in range(n)]


,x,t,Discharge_Sto_Nino,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Velocity_Sto_Nino,friction_coeff,slope
0,14420,0.0,21.033407,0,1,2,0,0,12.18,0.026165,0.033,0.000667
1,14420,3600.0,21.280072,0,1,1,1,0,12.19,0.026450,0.033,0.000667
2,14420,7200.0,21.280072,1,1,1,0,1,12.19,0.026450,0.033,0.000667
3,14420,10800.0,21.529056,0,0,0,1,0,12.20,0.026738,0.033,0.000667
4,14420,14400.0,21.529056,1,1,1,0,0,12.20,0.026738,0.033,0.000667


In [8]:
# Splitting for time series: split into 50-25-25
train_df = df[0:int(n*0.5)]
val_df = df[int(n*0.5):int(n*0.75)]
test_df = df[int(n*0.75):]

## Window Generator

In [9]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inpauts, labels
    
    # Creating tf datasets for more convenient use and integration into model in the future
    def make_dataset(self, data, shuffle=True):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=shuffle,
            batch_size=32,)

        ds = ds.map(self.split_window)

        return ds
    
    # properties to access them as tf datasets
    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df, False)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

In [10]:
# The wide window uses independent hours of data as input to predict the water level of the next hour
# Here, the prediction is done on 6 hours
# This is used for Dense and Recurrent Neural Networks
wide_window = WindowGenerator(
        input_width=6, label_width=6, shift=1,
        label_columns=['Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']
    )

wide_window

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [1 2 3 4 5 6]
Label column name(s): ['Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']

In [11]:
for example_inputs, example_labels in wide_window.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 6, 12)
Labels shape (batch, time, features): (32, 6, 2)


In [12]:
wide_window.example

(<tf.Tensor: shape=(32, 6, 12), dtype=float32, numpy=
 array([[[1.4420000e+04, 6.4692000e+06, 1.4817737e+01, ...,
          1.8882351e-02, 3.3000000e-02, 6.6666666e-04],
         [1.4420000e+04, 6.4728000e+06, 1.4817737e+01, ...,
          1.8882351e-02, 3.3000000e-02, 6.6666666e-04],
         [1.4420000e+04, 6.4764000e+06, 1.4817737e+01, ...,
          1.8882351e-02, 3.3000000e-02, 6.6666666e-04],
         [1.4420000e+04, 6.4800000e+06, 1.4817737e+01, ...,
          1.8882351e-02, 3.3000000e-02, 6.6666666e-04],
         [1.4420000e+04, 6.4836000e+06, 1.4817737e+01, ...,
          1.8882351e-02, 3.3000000e-02, 6.6666666e-04],
         [1.4420000e+04, 6.4872000e+06, 1.4817737e+01, ...,
          1.8882351e-02, 3.3000000e-02, 6.6666666e-04]],
 
        [[1.4420000e+04, 4.6620000e+06, 1.5003956e+01, ...,
          1.9103585e-02, 3.3000000e-02, 6.6666666e-04],
         [1.4420000e+04, 4.6656000e+06, 1.5003956e+01, ...,
          1.9103585e-02, 3.3000000e-02, 6.6666666e-04],
         [1.442

In [13]:
# The conv window is used for the Convolutional Neural Netwrok
# 6 consecutive hours of data are used together to make predictions one hour into the future
CONV_WIDTH = 6
conv_window = WindowGenerator(
        input_width=CONV_WIDTH,
        label_width=1,
        shift=1,
        label_columns=['Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']
    )

conv_window

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [6]
Label column name(s): ['Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']

In [14]:
for example_inputs, example_labels in conv_window.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 6, 12)
Labels shape (batch, time, features): (32, 1, 2)


## Metrics

In [15]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

In [16]:
def NSE(y_true, y_pred):
    '''
    This is the Nash-Sutcliffe Efficiency Coefficient
    '''
    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

## Training Loop

In [17]:
# For easy compiling and fitting of different models
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(), 
        optimizer='adam', 
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )

    history = model.fit(
        window.train, 
        epochs=MAX_EPOCHS,
        validation_data=window.val,
        callbacks=[early_stopping]
    )

    return history

## Models

In [18]:
# Dense Neural Network
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
])

# Convolution Neural Network
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=(CONV_WIDTH,), activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2),
])

# LSTM
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
])

## Training

In [20]:
dense_history = compile_and_fit(dense, wide_window)

Epoch 1/20
274/274 [==============================] - 3s 5ms/step - loss: 4613106688.0000 - mean_squared_error: 4613105152.0000 - NSE: -119887912.0000 - r_square: 0.0799 - val_loss: 11783273.0000 - val_mean_squared_error: 11783271.0000 - val_NSE: -330772.1875 - val_r_square: 0.0348
Epoch 2/20
274/274 [==============================] - 1s 4ms/step - loss: 467107.3438 - mean_squared_error: 467107.3750 - NSE: -12166.2803 - r_square: 0.0851 - val_loss: 1433178.1250 - val_mean_squared_error: 1433178.1250 - val_NSE: -40215.1133 - val_r_square: 0.1028
Epoch 3/20
274/274 [==============================] - 1s 4ms/step - loss: 314777.0000 - mean_squared_error: 314777.1562 - NSE: -8212.3652 - r_square: 0.0690 - val_loss: 892901.5000 - val_mean_squared_error: 892901.1250 - val_NSE: -25055.0762 - val_r_square: 0.0599
Epoch 4/20
274/274 [==============================] - 1s 4ms/step - loss: 1359932.8750 - mean_squared_error: 1359933.2500 - NSE: -35407.8633 - r_square: 0.1449 - val_loss: 1778162.5000

In [21]:
conv_history = compile_and_fit(conv_model, conv_window)

Epoch 1/20
274/274 [==============================] - 2s 4ms/step - loss: 8399029248.0000 - mean_squared_error: 8399029248.0000 - NSE: -219512112.0000 - r_square: 0.1910 - val_loss: 628030.3750 - val_mean_squared_error: 628030.3750 - val_NSE: -17625.9629 - val_r_square: 0.2538
Epoch 2/20
274/274 [==============================] - 1s 4ms/step - loss: 24056002.0000 - mean_squared_error: 24056002.0000 - NSE: -621302.8750 - r_square: 0.2853 - val_loss: 177304.4844 - val_mean_squared_error: 177304.4844 - val_NSE: -4975.5845 - val_r_square: 0.2218
Epoch 3/20
274/274 [==============================] - 1s 4ms/step - loss: 98119.4688 - mean_squared_error: 98119.4688 - NSE: -2557.9043 - r_square: 0.2443 - val_loss: 98930.2422 - val_mean_squared_error: 98930.2422 - val_NSE: -2775.3674 - val_r_square: 0.1983
Epoch 4/20
274/274 [==============================] - 1s 4ms/step - loss: 43933.2461 - mean_squared_error: 43933.2461 - NSE: -1146.5483 - r_square: 0.2170 - val_loss: 125979.6953 - val_mean_sq

In [22]:
lstm_history = compile_and_fit(lstm_model, wide_window)

Epoch 1/20
274/274 [==============================] - 6s 10ms/step - loss: 8.5230 - mean_squared_error: 8.5230 - NSE: 0.7786 - r_square: 0.0014 - val_loss: 0.1899 - val_mean_squared_error: 0.1899 - val_NSE: 0.9947 - val_r_square: 1.2137e-14
Epoch 2/20
274/274 [==============================] - 2s 8ms/step - loss: 0.2662 - mean_squared_error: 0.2662 - NSE: 0.9931 - r_square: 0.0013 - val_loss: 0.1399 - val_mean_squared_error: 0.1399 - val_NSE: 0.9961 - val_r_square: 2.7700e-15
Epoch 3/20
274/274 [==============================] - 2s 7ms/step - loss: 0.2391 - mean_squared_error: 0.2391 - NSE: 0.9938 - r_square: 4.3575e-05 - val_loss: 0.1272 - val_mean_squared_error: 0.1272 - val_NSE: 0.9964 - val_r_square: 1.1549e-14
Epoch 4/20
274/274 [==============================] - 2s 8ms/step - loss: 0.2333 - mean_squared_error: 0.2333 - NSE: 0.9940 - r_square: 5.1096e-05 - val_loss: 0.1507 - val_mean_squared_error: 0.1507 - val_NSE: 0.9958 - val_r_square: 1.5251e-14
Epoch 5/20
274/274 [===========

## Predictions

In [20]:
test_df

,x,t,Discharge_Sto_Nino,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Velocity_Sto_Nino,friction_coeff,slope
13140,14420,47390400.0,14.817736,0,0,0,0,0,11.89,0.018882,0.033,0.000667
13141,14420,47394000.0,14.450867,0,0,0,0,0,11.87,0.018446,0.033,0.000667
13142,14420,47397600.0,14.091331,0,0,0,0,0,11.85,0.018017,0.033,0.000667
13143,14420,47401200.0,13.914278,0,4,0,0,0,11.84,0.017806,0.033,0.000667
13144,14420,47404800.0,13.914278,11,19,0,22,0,11.84,0.017806,0.033,0.000667
...,...,...,...,...,...,...,...,...,...,...,...,...
17515,14420,63140400.0,28.244204,0,0,0,0,0,12.44,0.034401,0.033,0.000667
17516,14420,63144000.0,28.244204,0,0,0,0,0,12.44,0.034401,0.033,0.000667
17517,14420,63147600.0,28.244204,0,0,0,0,0,12.44,0.034401,0.033,0.000667
17518,14420,63151200.0,28.244204,0,0,0,0,0,12.44,0.034401,0.033,0.000667


In [23]:
y_true = test_df[['Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']]
y_true = y_true.values
y_true

array([[11.89      ,  0.01888235],
       [11.87      ,  0.01844587],
       [11.85      ,  0.0180173 ],
       ...,
       [12.44      ,  0.03440052],
       [12.44      ,  0.03440052],
       [12.44      ,  0.03440052]])

In [24]:
y_true_h = y_true[1:,0]
y_true_v = y_true[1:,1]

In [25]:
y_true_h

array([11.87, 11.85, 11.84, ..., 12.44, 12.44, 12.44])

In [26]:
y_true_h.shape

(4379,)

In [27]:
y_true_v

array([0.01844587, 0.0180173 , 0.01780595, ..., 0.03440052, 0.03440052,
       0.03440052])

In [28]:
y_true_v.shape

(4379,)

In [29]:
y_pred_h = []
y_pred_v = []
batch_num = 1

for x, y in wide_window.test:
    predictions = lstm_model(x)
    
    if batch_num == 1:
        for i in range(len(predictions[0])):
            y_pred_h.append(predictions[0][i][0])
            y_pred_v.append(predictions[0][i][1])
            
        for i in range(1, len(predictions)):
            y_pred_h.append(predictions[i][-1][0])
            y_pred_v.append(predictions[i][-1][1])
            
        batch_num += 1
        
    else:
        for i in range(len(predictions)):
            y_pred_h.append(predictions[i][-1][0])
            y_pred_v.append(predictions[i][-1][1])
            
y_pred_h = np.array(y_pred_h)
y_pred_v = np.array(y_pred_v)

In [50]:
np.set_printoptions(suppress=True)

In [51]:
for x, y in wide_window.test.take(1):
    print(x[:10])
    print(lstm_model(x[:10]))
    print(y[:10])

tf.Tensor(
[[[   14420.         47390400.               14.817737
          0.                0.                0.
          0.                0.               11.89
          0.01888235        0.033             0.00066667]
  [   14420.         47394000.               14.450867
          0.                0.                0.
          0.                0.               11.87
          0.01844587        0.033             0.00066667]
  [   14420.         47397600.               14.0913315
          0.                0.                0.
          0.                0.               11.85
          0.0180173         0.033             0.00066667]
  [   14420.         47401200.               13.914278
          0.                4.                0.
          0.                0.               11.84
          0.01780595        0.033             0.00066667]
  [   14420.         47404800.               13.914278
         11.               19.                0.
         22.                0.  

In [30]:
y_pred_h

array([12.558904, 12.544382, 12.582944, ..., 12.589076, 12.589076,
       12.589076], dtype=float32)

In [31]:
y_pred_h.shape

(4379,)

In [32]:
y_pred_v

array([0.03096219, 0.03140775, 0.03177941, ..., 0.03183882, 0.03183882,
       0.03183882], dtype=float32)

In [33]:
y_pred_v.shape

(4379,)

In [34]:
SS_res_h = np.sum(np.square(y_true_h - y_pred_h)) 
SS_tot_h = np.sum(np.square(y_true_h - np.mean(y_true_h))) 

nse_h =  1 - SS_res_h/SS_tot_h
nse_h

-0.007145291529500719

In [35]:
SS_res_v = np.sum(np.square(y_true_v - y_pred_v)) 
SS_tot_v = np.sum(np.square(y_true_v - np.mean(y_true_v))) 

nse_v =  1 - SS_res_v/SS_tot_v
nse_v

-0.1521232435810489

In [36]:
mse_h = np.mean(np.square(y_true_h-y_pred_h))
mse_h

0.3126408270726993

In [37]:
mse_v = np.mean(np.square(y_true_v-y_pred_v))
mse_v

0.0019454772494940883

In [38]:
np.savetxt('multivariate_lstm_window_predictions.csv', y_pred_h, delimiter=',', fmt='%.4f')